In [ ]:
import numpy 


class Sudoku:
    def __init__(self):
        self.sudoku_temp = []
        self.solved_sudoku = []
        self.grid = []
#         self.cycles = 0
        self.counts = dict()


    #check to see if value in cell is possible under constraints
    def possible(self,y, x, n):
        
        # check row/column for number
        if n in self.sudoku_temp[:][y]:
            return False
        if n in self.sudoku_temp[:, x]:
            return False
        
        # check block
        for i in range(3):
            for j in range(3):
                if self.sudoku_temp[(y // 3) * 3 + i][(x // 3) * 3 + j] == n:
                    return False
        return True


    def check_if_valid(self,sudoku_line):
        list1 = np.arange(1, 10)
        return all(int(elem) in sudoku_line for elem in list1)

    #Check Sudoku
    def checker(self):
        # Check board has iterated
        if len(self.solved_sudoku) == 0:
            #print('BOARD WRONG')
            #print(self.solved_sudoku)
            return True

        board = self.solved_sudoku.copy()
        
        # Check Row
        for i in range(9):
            if not self.check_if_valid(board[i]):
                #print('ROW WRONG')
                return True
            # Check Column
            cols = np.transpose(board)
            if not self.check_if_valid(cols[i]):
                #print('COLUMN WRONG')
                return True
        # Check Block
        square_size = 3
        for i in range(9):
            square_r, square_c = divmod(i, square_size)
            square = board[square_r * square_size:(square_r + 1) * square_size,
                    square_c * square_size:(square_c + 1) * square_size]
            if not self.check_if_valid(square.reshape(9)):
                #print('BLOCK WRONG')
                return True
            
        return False


    def pre_processing(self):
        #count zeros
        
        for i in self.sudoku_temp:
            for ii in i:
                self.counts[ii] = self.counts.get(ii, 0) + 1 
        counts=sorted(self.counts, key=self.counts.get, reverse=True)
        
        #build grid
        self.grid = [[y, x] for x in range(9) for y in range(9) if self.sudoku_temp[x][y]==0]
        """
        Cell[0] => Column
        Cell[1] => Row
        Cell[2] => Possible Entries for Cell
                   Sorted by the values with the most numbers on the board first.
                   e.g If there are more 9's on the board than any other number, 9's will be trialled first on each iteration
                   (as long as 9 is a possible number)
        However if the board is over 2/3rds 0's (arbitrary), then the grid is sorted so the cells with the least possible values are trialled first.
        """
        remove_list = []
        for cell in self.grid:
            
            #find numbers in the block
            cellnumbers = [self.sudoku_temp[(cell[1] // 3) * 3 + i][(cell[0] // 3) * 3 + j] for i in range(3) for j in range(3)]

            #if there are any rows with only one solution they are filled in
            cellnumbers_row = self.sudoku_temp[:][cell[1]]
            if np.count_nonzero(cellnumbers_row==0)==1:
                self.sudoku_temp[cell[1]][np.where(cellnumbers_row == 0)[0][0]] = [i for i in range(1, 10) if i not in cellnumbers_row][0]
                remove_list.append(cell)
                continue
            
            #if there are any columns with only one solution they are filled in
            cellnumbers_col = self.sudoku_temp[:, cell[0]]
            if np.count_nonzero(cellnumbers_col==0)==1:
                self.sudoku_temp[np.where(cellnumbers_col == 0)[0][0],cell[0]] = [i for i in range(1, 10) if i not in cellnumbers_col][0]
                remove_list.append(cell)
                continue
                
            #find possible values for each cell
            nums = list(set(np.concatenate((cellnumbers_row, cellnumbers_col, cellnumbers), axis=None)))
            nums = [i for i in range(1, 10) if i not in [int(num) for num in nums]]
            if sorted(self.counts.items())[0][1]%60!=0:
                nums = [i for i in counts if i in nums]
                nums_ordered = set()
                for x in nums:
                    nums_ordered.add(x)
                cell += [list(nums_ordered)]
            else:
                cell += [list(nums)]
            if len(cell[2])==1:
                self.sudoku_temp[cell[1]][cell[0]]=cell[2][0]
                remove_list.append(cell)
                continue
        

            
        #remove cells that have just been filled and sort grid
        self.grid = [i for i in self.grid if i not in remove_list]
        if sorted(self.counts.items())[0][1]>=60 and sorted(self.counts.items())[1][0]==1 :
            self.grid = sorted(self.grid, key=lambda x: len(x[2]), reverse=False)
        
        #if any are removed, pre_process again. Not optimised but cuts some time down for the beefier sudokus
        if len(remove_list)>0:
            self.counts = dict()
            self.pre_processing()
 
    #Solve Sudoku
    def solver(self):                 
        for cell in self.grid:
            if self.sudoku_temp[cell[1]][cell[0]] == 0:
                for n in cell[2]:
                    if self.possible(cell[1], cell[0], n):
                        self.sudoku_temp[cell[1]][cell[0]] = n
                        #self.cycles+=1
                        if self.solver():
                            return True
                        else:
                            self.sudoku_temp[cell[1]][cell[0]] = 0
                return False        
        self.solved_sudoku = self.sudoku_temp.copy()
        return True


    def _sudoku_solver_(self,sudoku_init):
        self.sudoku_temp = sudoku_init
        for i in self.sudoku_temp:
            self.zeros += list(i).count(0)
        
        self.pre_processing()
        #self.pre_processing()
        
        #check if solved from pre_processing
        if len(self.grid)!=0:
            self.solver()
        else:
            self.solved_sudoku = self.sudoku_temp.copy()
            
        #check answer
        test = self.checker()
        if test:
            return np.full((9, 9), -1.)#, self.cycles,self.zeros
        else:
            return self.solved_sudoku#, self.cycles,self.zeros
        
def sudoku_solver(initial_sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """
    instance=Sudoku()
    return instance._sudoku_solver_(initial_sudoku)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# ax1 = sns.set_style(style=None, rc=None )
ax1 = sns.set_theme(style="ticks")
fig, ax1 = plt.subplots(figsize=(12,6))

ax2 = ax1.twinx()




# ax = sns.barplot(data=a, x='Scenario', y='Duration', hue='Program', palette=palette)
sns.set_color_codes("pastel")
g= sns.barplot(x="Sudoku", y="Missing Values", data=df,
            label="Zeros", palette=palette,ax=ax1)

sns.set_color_codes("muted")
sns.lineplot(x="Sudoku", y="Time", data=df,
            label="Time (seconds)", color="k" ,ax=ax2)
g.set_xticklabels(labels =df["Sudoku"], rotation=90)
#sns.set_xticklabels(90)
ax2.grid(False)
plt.show()

In [ ]:
df2 = df.loc[df['Difficulty'] == 'hard']
df2 = df2.sort_values(by=['Time'])
df2

ax1 = sns.set_theme(style="ticks")
fig, ax1 = plt.subplots(figsize=(12,6))

ax2 = ax1.twinx()




# ax = sns.barplot(data=a, x='Scenario', y='Duration', hue='Program', palette=palette)
sns.set_color_codes("pastel")
g= sns.barplot(x="Sudoku", y="Missing Values", data=df2,
            label="Zeros", palette=palette,ax=ax1)

sns.set_color_codes("muted")
g2=sns.lineplot(x="Sudoku", y="Time", data=df2,
            label="Time (seconds)", color="k" ,ax=ax2)
g.set_xticklabels(labels =df["Sudoku"], rotation=90)
g2.set(ylabel='Time (s)')
#sns.set_xticklabels(90)
ax2.grid(False)
ax1.set(ylim=(55, 61))
plt.show()

In [ ]:
sns.regplot(x="Missing Values", y="Iterations", data=df2,logx=True)
plt.show()

my_y = df["Iterations"]
my_x = df["Missing Values"]

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(my_x, my_y)
print(slope,intercept,r_value**2,std_err,p_value)

In [ ]:
import scipy

my_y = df['Iterations']
my_x = df['Time']

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(my_x, my_y)
    
print(slope,intercept,r_value**2,std_err)
equation = "y = "+str(round(round(slope,-1)))+"x+"+str(round(round(intercept,-1)))
g=sns.lmplot(x="Time", y="Iterations", data=df)
g.fig.text(0.28, 0.9,equation, fontsize=9)
g.set(xlabel='Time (s)')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(6, 15))

# Load the example car crash dataset
crashes = sns.load_dataset("car_crashes").sort_values("total", ascending=False)

# Plot the total crashes
sns.set_color_codes("pastel")
sns.barplot(x="total", y="abbrev", data=crashes,
            label="Total", color="b")

# Plot the crashes where alcohol was involved
sns.set_color_codes("muted")
sns.barplot(x="alcohol", y="abbrev", data=crashes,
            label="Alcohol-involved", color="b")

# Add a legend and informative axis label
ax.legend(ncol=3, loc="lower right", frameon=True)
ax.set(xlim=(0, 24), ylabel="",
       xlabel="Automobile collisions per billion miles")
sns.despine(left=True, bottom=True)
crashes